In [90]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib as plot
import math
import talib
from sklearn import metrics, linear_model, preprocessing, svm, model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
conn = sqlite3.connect('poloniex_0.1.db')
c = conn.cursor()

In [86]:
df = pd.read_sql_query("SELECT * FROM candles_USDT_BTC", conn)
df = df.drop('id' ,1)
df = df.drop('trades' ,1)
df['start'] = pd.to_datetime(df['start'],unit='s')
df.set_index('start' , inplace = True)
#df.index.dtype
df.head(10)
#print (len(df.index))


,open,high,low,close,vwp,volume
start,,,,,,
2017-06-11 19:51:00,2923.996734,2932.600000,2923.996734,2925.828930,2929.711168,10.337254
2017-06-11 19:52:00,2930.373888,2932.600000,2925.828930,2925.996734,2928.910755,2.599433
2017-06-11 19:53:00,2932.600000,2932.944889,2925.996734,2930.000000,2930.652682,7.102010
2017-06-11 19:54:00,2932.944889,2935.107505,2923.494110,2923.996834,2930.371417,9.232048
2017-06-11 19:55:00,2923.996834,2935.000000,2923.996734,2923.996734,2928.012814,1.390456
2017-06-11 19:56:00,2932.835523,2932.835523,2923.996734,2923.996734,2924.764786,3.015836
2017-06-11 19:57:00,2923.996735,2930.528568,2923.996735,2930.528568,2924.291140,1.462843
2017-06-11 19:58:00,2923.996735,2932.835522,2923.996735,2930.548878,2931.113139,6.586996
2017-06-11 19:59:00,2930.548878,2932.835522,2924.416129,2928.274152,2927.959137,0.607046


In [43]:
def safediv(x,y):
    if y == 0 :
        return np.nan
    else:
        return x/y

In [48]:
def ohlcsum(df):
    #df = df.sort()
    
    return {
       'open': df['open'][0],
       'high': df['high'].max(),
       'low': df['low'].min(),
       'close': df['close'][-1],
       'volume': df['volume'].sum(),
        
        'vwp': safediv((df['vwp']* df['volume']).sum(),df['volume'].sum()) 
        
      }

In [49]:
def convertcandles(df, mins):
    s = str(mins) + 'T'
    return( df.resample(s).agg(ohlcsum))
    
    
#convertcandles(df, 120)
    

In [98]:
newdf = convertcandles(df,15)

#newdf = df
price = np.array(newdf['vwp'])
#sma  = talib.SMA(price,50)
m = talib.MACD(price, 12,26,9)

mom = talib.MOM(price,12 )
rsi = talib.RSI(price)
srsi = talib.STOCHRSI(price)
newdf['srsi'] = srsi
#m[1][40:60]
# newdf['msig'] = np.where((m[1]>m[2]),1,0)
newdf['msig'] = m[1]-m[2]
#newdf['mom'] = np.where(mom>0,1,0)
# newdf['rsi'] = np.where((rsi>50)& (rsi> np.roll(rsi,1)),1,0)
newdf['rsi'] = rsi
# newdf['sma'] = np.where((pd.rolling_mean(newdf['vwp'],9)- pd.rolling_mean(newdf['vwp'],26))>0,1,0)
newdf['target'] = np.where((newdf['close'].shift(1)- newdf['close'])>0,1,0)
newdf.dropna(axis = 0, how = 'any', inplace = True)
newdf.drop(newdf.index[0], inplace = True)
newdf.head(30)


ValueError: Length of values does not match length of index

In [88]:
factors = ['volume','msig','rsi']
X = newdf[factors]
Y = newdf['target']

In [96]:
Xtrain,  Xtest, Ytrain, Ytest = model_selection.train_test_split(X,Y)
#Xtrain.head()
scaler = preprocessing.MinMaxScaler(copy = False)
#Xtrain[['volume','rsi']]=scaler.fit_transform(Xtrain[['volume','rsi']])
mod = linear_model.LogisticRegression(solver = "saga")
mod.fit(Xtrain, Ytrain)
#Xtest[['volume','rsi']]=scaler.fit_transform(Xtest[['volume','rsi']])
print('LogisticRegression score: %f'
      % mod.score(Xtest, Ytest))

LogisticRegression score: 0.506522


In [97]:
# names = [ "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes"]
names = [ "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost"
         ]
# classifiers = [SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1),
#     AdaBoostClassifier(),
#     GaussianNB()]
classifiers = [SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]
for name, clf in zip(names, classifiers):
    clf.fit(Xtrain, Ytrain)
    score = clf.score(Xtest, Ytest)
    print(name, score)

('Linear SVM', 0.52826086956521734)
('RBF SVM', 0.50869565217391299)
('Decision Tree', 0.53913043478260869)
('Random Forest', 0.52391304347826084)
('Neural Net', 0.50652173913043474)
('AdaBoost', 0.56086956521739129)
